In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logistic_regression').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
my_data = spark.read.format('libsvm').load('data/sample_libsvm_data.txt')
print(my_data.count(), len(my_data.columns))

100 2


In [8]:
my_data.head(1)

[Row(label=0.0, features=SparseVector(692, {127: 51.0, 128: 159.0, 129: 253.0, 130: 159.0, 131: 50.0, 154: 48.0, 155: 238.0, 156: 252.0, 157: 252.0, 158: 252.0, 159: 237.0, 181: 54.0, 182: 227.0, 183: 253.0, 184: 252.0, 185: 239.0, 186: 233.0, 187: 252.0, 188: 57.0, 189: 6.0, 207: 10.0, 208: 60.0, 209: 224.0, 210: 252.0, 211: 253.0, 212: 252.0, 213: 202.0, 214: 84.0, 215: 252.0, 216: 253.0, 217: 122.0, 235: 163.0, 236: 252.0, 237: 252.0, 238: 252.0, 239: 253.0, 240: 252.0, 241: 252.0, 242: 96.0, 243: 189.0, 244: 253.0, 245: 167.0, 262: 51.0, 263: 238.0, 264: 253.0, 265: 253.0, 266: 190.0, 267: 114.0, 268: 253.0, 269: 228.0, 270: 47.0, 271: 79.0, 272: 255.0, 273: 168.0, 289: 48.0, 290: 238.0, 291: 252.0, 292: 252.0, 293: 179.0, 294: 12.0, 295: 75.0, 296: 121.0, 297: 21.0, 300: 253.0, 301: 243.0, 302: 50.0, 316: 38.0, 317: 165.0, 318: 253.0, 319: 233.0, 320: 208.0, 321: 84.0, 328: 253.0, 329: 252.0, 330: 165.0, 343: 7.0, 344: 178.0, 345: 252.0, 346: 240.0, 347: 71.0, 348: 19.0, 349: 28.0

In [6]:
my_data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows



In [15]:
my_log_reg_model = LogisticRegression()
fitted_logreg = my_log_reg_model.fit(my_data)
log_summary = fitted_logreg.summary

In [18]:
log_summary.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000486...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716171...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298141...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113070...|       1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [16]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



### Train

In [19]:
lr_train, lr_test = my_data.randomSplit([0.7,0.3])
final_model = LogisticRegression()
fit_final = final_model.fit(lr_train)
prediction_and_labels = fit_final.evaluate(lr_test)
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[19.1866615730400...|[0.99999999535122...|       0.0|
|  0.0|(692,[123,124,125...|[34.1546542675934...|[0.99999999999999...|       0.0|
|  0.0|(692,[124,125,126...|[5.38447577945173...|[0.99543370230379...|       0.0|
|  0.0|(692,[124,125,126...|[28.4852460167573...|[0.99999999999957...|       0.0|
|  0.0|(692,[126,127,128...|[15.3896597895831...|[0.99999979281629...|       0.0|
|  0.0|(692,[126,127,128...|[22.0360674871928...|[0.99999999973093...|       0.0|
|  0.0|(692,[126,127,128...|[29.9414762922212...|[0.99999999999990...|       0.0|
|  0.0|(692,[127,128,129...|[17.1644118660348...|[0.99999996487707...|       0.0|
|  0.0|(692,[150,151,152...|[21.8376521338909...|[0.99999999967188...|       0.0|
|  0.0|(692,[153

In [21]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, MulticlassClassificationEvaluator)

In [22]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='label')
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)
my_final_roc

1.0